In [1]:
import os
import numpy as np
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle, class_weight
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
import tensorflow_addons as tfa
from tensorflow.keras.regularizers import l2

/Users/amartinh/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [8]:
base_training_path = "/Volumes/HD710PRO/Fire_and_Hurricane_Images/Fire/BinaryData/Training"
base_testing_path = "/Volumes/HD710PRO/Fire_and_Hurricane_Images/Fire/BinaryData/Test"

#custom image generator
def custom_image_generator(file_paths, batch_size):
    while True:
        file_paths = shuffle(file_paths)
        for i in range (0, len(file_paths), batch_size):
            batch_files = file_paths[i:i + batch_size]
            images, labels =[], []
            for file in batch_files:
                with rasterio.open(file) as src:
                    image = src.read()
                    image = np.moveaxis(image, 0, -1)
                label = 1 if '/Damaged/' in file else 0
                images.append(image)
                labels.append(label)
            yield np.array(images), np.array(labels)

In [9]:

# Function to get file paths
def get_file_paths(base_path):
    file_paths = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.tif'):
                file_paths.append(os.path.join(root, file))
    return file_paths

# Get file paths
train_files = get_file_paths(base_training_path)
test_files = get_file_paths(base_testing_path)

# Split training data for validation
train_files, val_files = train_test_split(train_files, test_size=0.1, random_state=42)


In [10]:
# Compute class weights
labels = [1 if '/Damaged/' in file else 0 for file in train_files]
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = dict(enumerate(class_weights))

batch_size = 32
train_generator = custom_image_generator(train_files, batch_size)
val_generator = custom_image_generator(val_files, batch_size)
test_generator = custom_image_generator(test_files, batch_size)

# Model architecture with Dropout and L2 regularization
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
for layer in base_model.layers:
    layer.trainable = False
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)  # L2 regularization
x = Dropout(0.5)(x)  # Dropout
output = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001))(x)  # L2 regularization on output layer as well
model = Model(inputs=base_model.input, outputs=output)


58889256/58889256 [==============================] - 6s 0us/step


In [11]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall(), tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

# Callbacks for early stopping, model checkpointing, and logging
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
csv_logger = CSVLogger('training_log.csv', append=True)

# Fit the model with callbacks
history = model.fit(train_generator,
                    steps_per_epoch=len(train_files) // batch_size,
                    validation_data=val_generator,
                    validation_steps=len(val_files) // batch_size,
                    epochs=10,
                    class_weight=class_weights_dict,
                    callbacks=[model_checkpoint_callback, early_stopping_callback, csv_logger])

# Save the model
model.save('wildfiredetectionafterVGG16.keras')

# Evaluate the model
test_loss, test_accuracy, test_precision, test_recall, test_f1_score = model.evaluate(test_generator, steps=len(test_files) // batch_size)
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1_score}")

Epoch 1/10
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2688/2688 [==============================] - 12450s 5s/step - loss: 4.1503 - accuracy: 0.7133 - precision: 0.4822 - recall: 0.6413 - f1_score: 0.5505 - val_loss: 1.6809 - val_accuracy: 0.8010 - val_precision: 0.6277 - val_recall: 0.6672 - val_f1_score: 0.6468
Epoch 2/10
2688/2688 [==============================] - 11629s 4s/step - loss: 1.5900 - accuracy: 0.7812 - precision: 0.5865 - recall: 0.6813 - f1_score: 0.6304 - val_loss: 1.5689 - val_accuracy: 0.8124 - val_precision: 0.6618 - val_recall: 0.6412 - val_f1_score: 0.6513
Epoch 3/10
2688/2688 [==============================] - 11778s 4s/step - loss: 1.2426 - accuracy: 0.8015 - precision: 0.6259 - recall: 0.6852 - f1_score: 0.6542 - val_loss: 1.1103 - val_accuracy: 0.7465 - val_precision: 0.5240 - val_recall: 0.7967 - val_f1_

/Users/amartinh/opt/anaconda3/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


331/331 [==============================] - 1201s 4s/step - loss: 0.7435 - accuracy: 0.7462 - precision: 0.5236 - recall: 0.8219 - f1_score: 0.6397
Test Accuracy: 0.7462235689163208
Test Precision: 0.523590087890625
Test Recall: 0.8219083547592163
Test F1 Score: [0.6396783]
